In [ ]:
import dt_apriltags
import pyrealsense2 as rs
import numpy as np
import cv2

In [ ]:
# Load json file
json_path = '/home/commaai-03/Test/test.json'
with open(json_path, 'r') as f:
    json_obj = json.load(f)
    
json_str = str(json_obj).replace("'", '\"')

ctx = rs.context()                                                
devices = ctx.query_devices()                                     
#print('devices: ', devices)
advance_mode = rs.rs400_advanced_mode(devices[0])
advance_mode.load_json(json_str)

In [ ]:
at_detector = dt_apriltags.Detector(searchpath=['apriltags'],
                       families='tagStandard41h12',
                       nthreads=1,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

In [ ]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848, 480, rs.format.rgb8, 30)

pipeline.start(config)

profiles = pipeline.get_active_profile()
profile = profiles.get_stream(rs.stream.color).as_video_stream_profile()
intrinsics = profile.get_intrinsics()
camera_params = [intrinsics.fx,
                 intrinsics.fy,
                 intrinsics.ppx,
                 intrinsics.ppy]
tag_size = 0.14576

# skip some frames
for x in range(30):
    pipeline.wait_for_frames()

align_to = rs.stream.color
align = rs.align(align_to)
add = False


try:
    while 1:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        profile = aligned_frames.get_profile()
        color = aligned_frames.get_color_frame()
        depth = aligned_frames.get_depth_frame()
        if not color or not depth:
            continue
        
        d_img = np.asanyarray(depth.get_data())
        img = np.asanyarray(color.get_data())
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        tags = at_detector.detect(gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)
        if tags:
            print(tags)
        cv2.imshow('Tags', img[...,::-1])
        k = cv2.waitKey(1)
        if k == 27 or k == ord('q'):
            break
        
finally:
    pipeline.stop()
    cv2.destroyAllWindows()

### Check Apriltag

In [ ]:
import dt_apriltags
import numpy as np
import cv2

# Create apriltags Detector
at_detector = dt_apriltags.Detector(searchpath=['apriltags'],
                       families='tagStandard41h12',
                       nthreads=1,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

# Load image file to gray.
img_file = '/home/commaai-03/Data/color/apriltags/rs15_1.jpg'
gray = cv2.imread(img_file, 0)

tags = at_detector.detect(gray)

for tag in tags:
    _round = lambda x: int(round(x))
    points = [(_round(tag.center[0]), _round(tag.center[1])),
              (_round(tag.corners[0][0]), _round(tag.corners[0][1])),
              (_round(tag.corners[1][0]), _round(tag.corners[1][1])),
              (_round(tag.corners[2][0]), _round(tag.corners[2][1])),
              (_round(tag.corners[3][0]), _round(tag.corners[3][1]))
             ]
    
    for point in points:
        cv2.circle(gray, point, 1, (0,0,255),-1)
        
cv2. imshow('Test', gray)
k = cv2.waitKey()
if k == 27 or k == ord('q'):
    cv2.destroyAllWindows()
else:
    cv2.destroyAllWindows()

In [ ]:
import os

save_dir = '/home/commaai-03/Data/color/test'
img_dir = '/home/commaai-03/Data/color/apriltags'
files = os.listdir(img_dir)

def circle2save(img_file, save_dir):
    gray = cv2.imread(img_file, 0)
    tags = at_detector.detect(gray)
    for tag in tags:
        _round = lambda x: int(round(x))
        points = [(_round(tag.center[0]), _round(tag.center[1])),
                  (_round(tag.corners[0][0]), _round(tag.corners[0][1])),
                  (_round(tag.corners[1][0]), _round(tag.corners[1][1])),
                  (_round(tag.corners[2][0]), _round(tag.corners[2][1])),
                  (_round(tag.corners[3][0]), _round(tag.corners[3][1]))
                 ]

        for point in points:
            cv2.circle(gray, point, 1, (0,0,255),-1)
    basename = os.path.basename(img_file)
    filename = os.path.join(save_dir, basename)
    cv2.imwrite(filename, gray)

for file in files:
    if '.jpg' in file:
        img_file = os.path.join(img_dir, file)
        circle2save(img_file, save_dir)